In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=50
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel1=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# dx = 1km; Np = 50M; Nz = 95
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc') #***
parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc') #***
res='1km'; t_res='1min_95nz'; Np_str='50e6'

In [ ]:
#LOADING IN H5 VARIABLES
PROCESSING=False
PROCESSING=True

print('loading vars')
dir2=dir+'Project_Algorithms/Entrainment/job_out_3/'
dir3=dir+'Project_Algorithms/Entrainment/'

if PROCESSING==False:
    file_path=dir3+f'3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
elif PROCESSING==True:
    file_path=dir3+f'3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
        
ED_ds1 = xr.open_dataset(file_path, phony_dims='sort',engine='h5netcdf')  # or engine='netcdf4'
ED_ds1 = ED_ds1.rename({
    'phony_dim_0': 'time',
    'phony_dim_1': 'zh',
    'phony_dim_2': 'yh',
    'phony_dim_3': 'xh'
})

file_path = dir + 'Variable_Calculation/' + f'Eulerian_Binary_Array_{res}_{t_res}.h5'
A_ds1 = xr.open_dataset(file_path, phony_dims='sort',engine='h5netcdf')  # or engine='netcdf4'
A_ds1 = A_ds1.rename({
    'phony_dim_0': 'time',
    'phony_dim_1': 'zh',
    'phony_dim_2': 'yh',
    'phony_dim_3': 'xh'
})

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
#DOMAIN SUBSETTING
############################################################

#FINDING NEW LEFT T-BOUNDARY
dt=data1['time'][1].item()/1e9 #seconds per timestep
dhours=(dt/60**2) #hours per timestep
start_hour=4 #10:00 am
t_start=int(start_hour/dhours)
#FINDING NEW RIGHT T-BOUNDARY
end_hour=11 #5pm
t_end=int(end_hour/dhours)
#PRINTING
print(f't in {t_start}:{t_end}')


#FINDING NEW TOP Z-BOUNDARY
dzh=data1['zh']
zh_bottom=0; zh_top=np.where(dzh>=19)[0][0]
dzf=data1['zf']
zf_bottom=0; zf_top=np.where(dzf>=20)[0][0]
#PRINTING
print(f'zh_top at {zh_bottom}:{zh_top}')

#FINDING THE NEW LEFT X-BOUNDARY
ocean_percent=2/8
left_to_coast=data1['xh'][0]+(data1['xh'][-1]-data1['xh'][0])*ocean_percent
where_coast_xh=np.where(data1['xh']>=left_to_coast)[0][0]#-25
where_coast_xf=np.where(data1['xf']>=left_to_coast)[0][0]#-25
#FINDING THE NEW RIGHT X-BOUNDARY
right_fraction=80/100
xf=data1['xf']-data1['xf'][0]
Nxf=len(xf)
end_xf=np.where(xf>Nxf*right_fraction)[0][0]
xh=data1['xh']-data1['xh'][0]
Nxh=len(xh)
end_xh=np.where(xh>Nxh*right_fraction)[0][0]
#PRINTING
# print(f'x in {0}:{where_coast_xh-1} FOR SEA')
print(f'x in {where_coast_xh}:{end_xh} FOR LAND')


#SUBSETTING CODE
data1=data1.isel(time=slice(t_start,None),zh=slice(zh_bottom,zh_top),zf=slice(zf_bottom,zf_top),xh=slice(where_coast_xh,end_xh+1),xf=slice(where_coast_xf,end_xf+1))
A_ds1=A_ds1.isel(time=slice(t_start,None),zh=slice(zh_bottom,zh_top),xh=slice(where_coast_xh,end_xh+1))
ED_ds1=ED_ds1.isel(time=slice(t_start,None),zh=slice(zh_bottom,zh_top),xh=slice(where_coast_xh,end_xh+1))

In [ ]:
####################################
# SETUP

In [ ]:
################################
#JOB ARRAY SETUP
################################
#*#*
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if '1e6' in Np_str:
    num_jobs=60 #1M parcels
if '50e6' in Np_str:
    num_jobs=200 #50M parcels
num_slurm_jobs=60
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(data1['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust
# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

In [ ]:
def GetData(ED_ds1,start_job,end_job):
    #Indexing Array with JobArray
    ED_ds=ED_ds1.isel(time=slice(start_job,end_job))
    return ED_ds
# ED_ds=GetData(ED_ds1,start_job,end_job)

In [ ]:
def GetA(A_ds1,start_job,end_job):
    A_ds=A_ds1.isel(time=slice(start_job,end_job))
    A_g=A_ds['A_g'].data; A_c=A_ds['A_c'].data
    return A_g,A_c
# [A_g,A_c]=GetA(A_ds1,start_job,end_job)

In [ ]:
#constants
Cp=1004 #Jkg-1K-1
Cv=717 #Jkg-1K-1
Rd=Cp-Cv #Jkg-1K-1
eps=0.608

Lx=(data1['xf'][-1].item()-data1['xf'][0].item())*1000 #x length (m)
Ly=(data1['yf'][-1].item()-data1['yf'][0].item())*1000 #y length (m)
Np=len(parcel1['xh']) #number of lagrangian parcles
dt=(data1['time'][1]-data1['time'][0]).item()/1e9 #sec
dx=(data1['xf'][1].item()-data1['xf'][0].item())*1e3 #meters
dy=(data1['yf'][1].item()-data1['yf'][0].item())*1e3 #meters
xs=data1['xf'].values*1000
ys=data1['yf'].values*1000
zs=data1['zf'].values*1000

def zf(z):
    k=z #z is the # level of z
    out=data1['zf'].values[k]*1000
    
    return out
# def rho(x,y,z,t):
#     p=data1['prs'].isel(xh=x,yh=y,zh=z,time=t).item()
#     p0=101325 #Pa
#     theta=data1['th'].isel(xh=x,yh=y,zh=z,time=t).item()
#     T=theta*(p/p0)**(Rd/Cp)
#     qv=data1['qv'].isel(xh=x,yh=y,zh=z,time=t).item()
#     # Tv=T*(1+eps*qv)
#     Tv=T*(eps+qv)/(eps*(1+qv))
#     rho = p/(Rd*Tv)
#     out=rho
#     return out

def rho(x,y,z,rho_data_t):
    out=rho_data_t[z,y,x]
    return out
def m(t):
    rho_data_t=data1['rho'].isel(time=t).data
    
    m=0
    #triple sum
    for k in range(len(data1['zh'])):
        dz=(zf(k+1)-zf(k))
        for j in range(len(data1['yh'])):
            for i in range(len(data1['xh'])):
                rho_out=rho(i,j,k,rho_data_t)
                m+=rho_out*dz

    #triple sum
    out=m*dx*dy/Np
    return out

In [ ]:

#Calculate Mass Constant
# calculate='single_time'
# calculate=True
calculate=False

if calculate==True:
    Nt=len(data1['time'])
    m_arr=np.zeros((Nt))
    for t in np.arange(Nt):
        if np.mod(t,25)==0: print(t)
        m_arr[t]=m(t)
    dir3=dir+f'Project_Algorithms/Entrainment/'
    np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
elif calculate=='single_time':
    Nt=len(data1['time'])
    m_arr=np.zeros((Nt))

    t=0 #len(data1['time'])//2 #Pick some middle time
    m_300=m(t)
    for t in np.arange(Nt):
        m_arr[t]=m_300 #UNCOMMENT FOR FULL CALCULATION
    dir3=dir+f'Project_Algorithms/Entrainment/'
    np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
else:
    dir3=dir+f'Project_Algorithms/Entrainment/'
    m_arr = np.load(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy')

# # TESTING
# lst=[]
# for t in np.arange(133):
#     lst.append(m_arr[t])

# plt.plot(lst)
# (np.max(lst)-np.min(lst))*100/np.mean(lst)

In [ ]:
def apply_constant(profile_array,apply):
    if apply==True:
        Nt=profile_array.shape[0]
        Nz=profile_array.shape[1]
    
        profile_array/=(dx*dy*dt)
        for t in np.arange(Nt):
            profile_array[t]*=m_arr[t+index_adjust]
        for z in np.arange(Nz):
            dz=zf(z+1)-zf(z)
            profile_array[:,z]/=dz
    # print(dx,dy,dz,dt,m_arr[t+index_adjust],Nt,Nz) #TESTING
    return profile_array

In [ ]:
def CallVariables(ED_ds):
    
    profile_array_e_g = ED_ds["profile_array_e_g"].data
    profile_array_e_c = ED_ds["profile_array_e_c"].data
    profile_array_d_g = ED_ds["profile_array_d_g"].data
    profile_array_d_c = ED_ds["profile_array_d_c"].data
    
    # APPLY CONSTANTS TO ENTRAINMENT/DETRAINMENT VALUES
    profile_array_e_g = apply_constant(profile_array_e_g, apply=True)
    profile_array_e_c = apply_constant(profile_array_e_c, apply=True)
    profile_array_d_g = -apply_constant(profile_array_d_g, apply=True)
    profile_array_d_c = -apply_constant(profile_array_d_c, apply=True)
    profile_array_net_g = profile_array_e_g - profile_array_d_g
    profile_array_net_c = profile_array_e_c - profile_array_d_c

    VARs = {
        "profile_array_e_g": profile_array_e_g,
        "profile_array_e_c": profile_array_e_c,
        "profile_array_d_g": profile_array_d_g,
        "profile_array_d_c": profile_array_d_c,
        "profile_array_net_g": profile_array_net_g,
        "profile_array_net_c": profile_array_net_c
    }
    return VARs

In [ ]:
####################################
#Functions

In [ ]:
# Thresholds
w_thresh1 = 0.1
w_thresh2 = 0.5
qcqi_thresh = 1e-6

def DomainProfile(VARs,data_type, A_g,A_c):
    zhs = data1['zh'].values
    profiles = {}  # Store profiles for all variables

    # Initialize profiles for each variable
    for var in VARs:
        profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
        profiles[var][:, 2] = zhs

    # Threshold mask
    if data_type == "general":
        where_updraft = (A_g==True)
    elif data_type == "cloudy":
        where_updraft = (A_c==True)
    t_ind, z_ind, y_ind, x_ind = np.where(where_updraft)

    # Variable selection dictionary

    # Iterate over each variable in var_names and bin the data
    for var in VARs:
        masked_data = VARs[var][where_updraft]
        np.add.at(profiles[var][:, 0], z_ind, masked_data)
        np.add.at(profiles[var][:, 1], z_ind, 1)

        # if var=='profile_array_d_c': #TESTING
        #     print(1)
        #     print(np.where(where_updraft))
        #     print(z_ind)
        #     print(VARs[var][where_updraft])
        #     print(np.where(VARs[var][where_updraft]))

    return profiles
# profiles=DomainProfile(VARs,data_type,A_g,A_c)

In [ ]:
def SaveProfile(Dictionary, data_type, job_id):
    dir2=dir+'Project_Algorithms/Domain_Profiles/'
    if data_type == "general":
        output_file = dir2+f'job_out/general_eulerian_entrainment_profiles_{res}_{t_res}_{Np_str}'
    elif data_type == "cloudy":
        output_file = dir2+f'job_out/cloudy_eulerian_entrainment_profiles_{res}_{t_res}_{Np_str}'
    output_file+=f'_{job_id}.h5'
    
    with h5py.File(output_file, 'w') as f:
        for var in Dictionary:
            profile_var = Dictionary[var]
            f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
########################################
#Running

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array

print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}')
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

    #SLICING DATA
    ED_ds=GetData(ED_ds1,start_job,end_job)
    [A_g,A_c]=GetA(A_ds1,start_job,end_job)

    #GETTING VARIABLES DICTIONARY
    VARs=CallVariables(ED_ds)
    
    for data_type in ['general','cloudy']:
        #GETTING DATA AND PUTTING IN A DICTIONARY    
        Dictionary=DomainProfile(VARs,data_type,A_g,A_c)
        SaveProfile(Dictionary,data_type,job_id)
        # check_memory(globals())
        del Dictionary
    del VARs

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
# recombine=True

In [ ]:
def Recombine(num_jobs):
    dir2=dir+'Project_Algorithms/Domain_Profiles/'
    var_names = ['profile_array_e_g','profile_array_e_c',
             'profile_array_d_g','profile_array_d_c',
             'profile_array_net_g','profile_array_net_c']
    
    data_types=['general','cloudy']
    for data_type in data_types:
        #MAKING OUTPUT FILE PATH
        if data_type == "general":
            output_file = dir2+f'job_out/general_eulerian_entrainment_profiles_{res}_{t_res}_{Np_str}.h5' 
        elif data_type == "cloudy":
            output_file = dir2+f'job_out/cloudy_eulerian_entrainment_profiles_{res}_{t_res}_{Np_str}.h5'
        
        #MAKING PROFILES DICTIONARY
        zhs = data1['zh'].values
        profiles = {}  #  Store profiles for all variables
        for var in var_names:
            profiles[var] = np.zeros((len(zhs), 3))  # column 1: var, column 2: counter, column 3: list of zhs
            profiles[var][:, 2] = zhs 

        for job_id in np.arange(1,num_jobs+1):
            if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
            #CALLING IN DATA
            if data_type == "general":
                input_file = dir2+f'job_out/general_eulerian_entrainment_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5' 
            elif data_type == "cloudy":
                input_file = dir2+f'job_out/cloudy_eulerian_entrainment_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5'
    
            #COMPILING PROFILES
            with h5py.File(input_file, 'r') as f:
                for var in var_names:  
                    profiles[var][:,0:1+1]+=f[f'{var}'][:,0:1+1]
        
        #SAVING INTO FINAL FORM
        with h5py.File(output_file, 'w') as f:
            for var in profiles:
                profile_var = profiles[var]
                f.create_dataset(f'{var}', data=profile_var, compression="gzip")

In [ ]:
if recombine==True:
    Recombine(num_jobs=num_jobs)